## Input File Statistics 

This is used to compute the standard deviations of the residuals of the input files so that we have a good working 
estimate of the standard deviations to use in the project.  Basically it is just coordinate transformation and 
calculation of the standard deviation.  The values used in the project are basically the average rounded.  The second 
section involves looking at the rate of change of the radar measured velocity and yaw angle. 

In [2]:
import math
import csv
import numpy as np


def compute(filename):
    with open(filename, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for row in reader:
            if row[0] == 'R':
                rho = float(row[1])
                phi = float(row[2])
                rho_dot = float(row[3])
 
                px = rho * math.cos(phi)
                py = rho * math.sin(phi)
                vx = rho_dot * math.cos(phi)
                vy = rho_dot * math.sin(phi)

                px_gt = float(row[5])
                py_gt = float(row[6])
                vx_gt = float(row[7])
                vy_gt = float(row[8])

                rho_gt = math.sqrt(px_gt*px_gt + py_gt*py_gt)
                phi_gt = math.atan2(py_gt, px_gt)
                if rho_gt < 0.001:
                    rho_dot_gt = 0.0
                else:
                    rho_dot_gt = (px_gt*vx_gt + py_gt*vy_gt) / rho_gt

                pxs.append(px - px_gt)
                pys.append(py - py_gt)

                vxs.append(vx - vx_gt)
                vys.append(vy - vy_gt)

                rhos.append(rho - rho_gt)
                phis.append(phi - phi_gt)
                rhodots.append(rho_dot - rho_dot_gt)

            if row[0] == 'L':
                px = float(row[1])
                py = float(row[2])

                px_gt = float(row[4])
                py_gt = float(row[5])

                pxs.append(px - px_gt)
                pys.append(py - py_gt)

def standard_deviation(data):
    data = np.array(data)
    return np.std(data)

def print_stats(filename):
    print('File: ', filename)
    print('px: ', standard_deviation(pxs))
    print('py: ', standard_deviation(pys))    
    print('vx: ', standard_deviation(vxs))
    print('vy: ', standard_deviation(vys))
    print('rho: ', standard_deviation(rhos))
    print('phi: ', standard_deviation(phis))
    print('rho_dot: ', standard_deviation(rhodots))
    print('')


for f in ['../data/sample-laser-radar-measurement-data-1.txt','../data/sample-laser-radar-measurement-data-2.txt']:
    pxs, pys, vxs, vys, rhos, phis, rhodots = [], [], [], [], [], [], []
    compute(f)
    print_stats(f)


File:  ../data/sample-laser-radar-measurement-data-1.txt
px:  0.0550621983605
py:  0.0482489411357
vx:  1.32375092114
vy:  1.67907181529
rho:  0.101831322138
phi:  0.00103346009555
rho_dot:  0.106276974355

File:  ../data/sample-laser-radar-measurement-data-2.txt
px:  0.208049435877
py:  0.218665738265
vx:  0.150008299815
vy:  0.539014910297
rho:  0.197839534372
phi:  0.00177516722645
rho_dot:  0.0985115500912



While the above values work really well for the measurement standard deviations, the below, which estimates the process standard deviations by looking at changes in phi and rho dot with respect to time, is only an order of magnitude starting point.  This, coupled with testing it with the UKF code and adjusting it to minimize the RMSE, is where the values used come from.

In [3]:
import math
import csv
import numpy as np


def compute(filename):
    with open(filename, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        last_phi = 0.0
        last_rho_dot = 0.0
        last_time = 0.0
        for row in reader:
            if row[0] == 'R':
                time = float(row[4])
                dt = (time - last_time) / 1000000.0
                last_time = time
                
                phi = float(row[2])
                delta_phi = phi - last_phi
                last_phi = phi
                phis.append(delta_phi / dt)
                
                rho_dot = float(row[3])
                delta_rho_dot = rho_dot - last_rho_dot
                last_rho_dot = rho_dot
                rho_dots.append(delta_rho_dot / dt)

for f in ['../data/sample-laser-radar-measurement-data-1.txt','../data/sample-laser-radar-measurement-data-2.txt']:
    phis, rho_dots = [], []
    compute(f)
    print("File: ", f)
    print("Phi std: ", np.std(phis))
    print("Rho_dot std: ", np.std(rho_dots))
    print("")
    

File:  ../data/sample-laser-radar-measurement-data-1.txt
Phi std:  0.2100944561
Rho_dot std:  8.68929097026

File:  ../data/sample-laser-radar-measurement-data-2.txt
Phi std:  0.00990205014222
Rho_dot std:  0.283344181369

